In [1]:
import os
import glob
import random

## Helper function to generate file list for Tacotron2

In [2]:
def random_sample(used_path, all_path):
    all_path.remove(used_path)
    return random.sample(all_path, 1)[0]

## Generate dataset dicts

In [3]:
rootdir = '/data3/IEMOCAP/IEMOCAP_full_release'

def emo_script(wav_dir, trans_dir):
    trans_map = {}
    for sess_path in sorted(glob.glob(wav_dir)):
        sess = sess_path.split('/')[-1]
        trans_path = os.path.join(trans_dir, sess +'.txt')
        with open(trans_path, 'r') as file:
            while True:
                line = file.readline()
                if not line:
                    break
                if line[0] == 'S':
                    split_line = line.split(':')
                    transcript = split_line[-1][1:-2]
                    wav_file_name = split_line[0].split()[0]
                    wav_name = wav_file_name +'.wav'
                    try: # filter out wavname like Ses03F_impro06_MXX1
                        x = int(wav_file_name[-3:])
                    except:
                        continue
                    wav_path = str(os.path.join(sess_path, wav_name))
                    speaker_sess_id = wav_name.split('.')[0][:-3]
                    speaker_sess_path = str(os.path.join(sess_path, speaker_sess_id))
                    all_same_wav = glob.glob(speaker_sess_path + '*.wav')
                    sampled_path = random_sample(wav_path, all_same_wav)
                    trans_map[wav_path] = [transcript, sampled_path]
    return trans_map

def dict2list(test_dict, flag):
    output = []
    for key, value in test_dict.items():
        if flag == 0:
            s = key + '|' + value[0] + '\n'
        else:
            s = value[1] + '|' + 'nothing is here' + '\n'
        output.append(s)
    return output


# # Test Code
# wav_dir = os.path.join(rootdir, 'Session4', 'sentences/wav/*')
# trans_dir = os.path.join(rootdir, 'Session4', 'dialog/transcriptions')
# trans_dict = emo_script(wav_dir, trans_dir)


## Generating .txt files for train, val, test

In [4]:
session_list = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']

for session in session_list:
    wav_dir = os.path.join(rootdir, session, 'sentences/wav/*')
    trans_dir = os.path.join(rootdir, session, 'dialog/transcriptions')
    if session == 'Session4':
        val_dict = emo_script(wav_dir, trans_dir)
        val_output = dict2list(val_dict, 0)
        val_output_alt = dict2list(val_dict, 1)
        with open('filelists/IEMOCAP/val_filelist.txt', 'w') as file:
            file.writelines(val_output)
        with open('filelists/IEMOCAP/val_filelist_alt.txt', 'w') as file:
            file.writelines(val_output_alt)
    elif session == 'Session5':
        test_dict = emo_script(wav_dir, trans_dir)
        test_output = dict2list(test_dict, 0)
        test_output_alt = dict2list(test_dict, 1)
        with open('filelists/IEMOCAP/test_filelist.txt', 'w') as file:
            file.writelines(test_output)
        with open('filelists/IEMOCAP/test_filelist_alt.txt', 'w') as file:
            file.writelines(test_output_alt)
    else:
        train_dict = emo_script(wav_dir, trans_dir)
        train_output = dict2list(train_dict, 0)
        train_output_alt = dict2list(train_dict, 1)
        with open('filelists/IEMOCAP/train_filelist.txt', 'a') as file:
            file.writelines(train_output)
        with open('filelists/IEMOCAP/train_filelist_alt.txt', 'a') as file:
            file.writelines(train_output_alt)

## Quality Check

In [5]:
original_path = './filelists/IEMOCAP'
for ftype in ['train', 'val', 'test']:
    file_path = original_path + '/' + ftype + '_filelist.txt'
    alt_path = original_path + '/' + ftype + '_filelist_alt.txt'
    with open(file_path, 'r') as a, open(alt_path, 'r') as b:
        content_a = a.readlines()
        content_b = b.readlines()
        for line_a, line_b in zip(content_a, content_b):
            if line_a.split('|')[0].split('.')[0][-4] == line_b.split('|')[0].split('.')[0][-4] and line_a.split('|')[0] != line_b.split('|')[0]:
                continue
            else:
                print('==============================')
                print('something wrong')
                print('ftype', ftype)
                print(line_a.split('|')[0].split('.')[0][-4])
                print(line_b.split('|')[0].split('.')[0][-4])
                print(line_a.split('|')[0])
                print(line_b.split('|')[0])
                print('==============================')
print('Finish Checking')
    

Finish Checking


## Checking the number

In [6]:
num_lines_train = sum(1 for line in open('filelists/IEMOCAP/train_filelist.txt'))
num_lines_val = sum(1 for line in open('filelists/IEMOCAP/val_filelist.txt'))
num_lines_test = sum(1 for line in open('filelists/IEMOCAP/test_filelist.txt'))
num_lines_train_alt = sum(1 for line in open('filelists/IEMOCAP/train_filelist_alt.txt'))
num_lines_val_alt = sum(1 for line in open('filelists/IEMOCAP/val_filelist_alt.txt'))
num_lines_test_alt = sum(1 for line in open('filelists/IEMOCAP/test_filelist_alt.txt'))

print('Check the numbers!')
print(num_lines_train)
print(num_lines_val)
print(num_lines_test)
print('====================')
print(num_lines_train_alt)
print(num_lines_val_alt)
print(num_lines_test_alt)

Check the numbers!
5766
2103
2170
5766
2103
2170


In [7]:
# Train count:  5788
# Valid count:  2103
# Test count:  2194
# total count:  10085


## Playground

In [15]:
list(enumerate(zip(x, y)))
(0, ((1, 2), (0, 5)))

[(0, ((1, 2), (0, 5))), (1, ((4, 5), (2, 6))), (2, ((2, 3), (3, 1)))]

In [18]:
x = [(1,2), (4,5), (2,3)]
y = [(0,5), (2,6), (3,1)]
for i, value in enumerate(zip(x, y)):
    (x1,x2), (y1, y2) = value
    print(x1)
    print(x2)
    print(y1)
    print(y2)

1
2
0
5
4
5
2
6
2
3
3
1
